In [75]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import undetected_chromedriver
import pandas as pd
from time import sleep
from multiprocessing import Pool
from functools import partial
import os

In [76]:
def fetchMatch(matchID, driver):
    driver.get("https://csgostats.gg/match/" + str(matchID))
    
    mm = driver.find_element(By.XPATH,"//body/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]/div")
    if(mm.get_attribute("textContent").strip()[:20] == "Official Matchmaking"):
        players =  driver.find_elements( By.CLASS_NAME, 'player-link')
        match = []
        for player in players:

            playerstats = {}
            playerstats["Match ID"] = matchID
            playerstats["Player ID"] = player.get_attribute('href')[-17:-1]

            PCard = player.find_element(By.XPATH,"./..").find_element(By.XPATH,"./..")

            rank = PCard.find_elements(By.CLASS_NAME, "rank")
            if rank.__len__():
                playerstats["Rank"] = (rank[0].get_attribute("src"))[-6:-4].strip("/")

            rankupicon = PCard.find_elements(By.XPATH, "./td/div/div/span")

            if(rankupicon.__len__()):
                rankup = rankupicon[0].get_attribute("class")
                if rankup == "glyphicon glyphicon-chevron-up" :
                    playerstats["Rank Gain"] = 1

                elif rankup == "glyphicon glyphicon-chevron-down" :
                    playerstats["Rank Gain"] = -1

            else :
                playerstats["Rank Gain"] = 0        
            
            # print(stats["rank gain"])
            # teamScore = player.find_element(By.XPATH,"./..")
            # teamScore = teamScore.find_element(By.XPATH,"./..")
            team = PCard.find_element(By.XPATH,"./..")
            
            #teamScore = teamScore.find_element(By.ID,'match-scoreboard')
            #teamScore = teamScore.find_element(By.CLASS_NAME,'scoreboard')
            
            playerstats["Team"] = team.find_element(By.XPATH, "./tr/td/div").get_attribute("textContent").strip() [-1:]

            teams = team.find_element(By.XPATH,"./..")

            scores = teams.find_elements(By.XPATH, "./tbody/tr/td/div[2]")

            playerstats["Team 1 Score"] = scores[0].get_attribute("textContent").strip()
            playerstats["Team 2 Score"] = scores[1].get_attribute("textContent").strip()

            stats = PCard.find_elements(By.TAG_NAME, "td")

            # for stat in stats:
            #     print(stat.get_attribute("data-collapse"),stat.get_attribute("textContent").strip() )
            
            

            playerstats["K"] =                  stats[2].get_attribute("textContent").strip()
            playerstats["D"] =                  stats[3].get_attribute("textContent").strip()  
            playerstats["A"] =                  stats[4].get_attribute("textContent").strip()  
            playerstats["+/-"] =                stats[5].get_attribute("textContent").strip()  
            playerstats["K/D"] =                stats[6].get_attribute("textContent").strip()  
            playerstats["ADR"] =                stats[7].get_attribute("textContent").strip()  
            playerstats["HS%"] =                stats[8].get_attribute("textContent").strip()  
            playerstats["KAST"] =               stats[9].get_attribute("textContent").strip()  
            playerstats["Rating"] =             stats[10].get_attribute("textContent").strip()  
            playerstats["EF"] =                 stats[11].get_attribute("textContent").strip()  
            playerstats["FA"] =                 stats[12].get_attribute("textContent").strip()  
            playerstats["EBT"] =                stats[13].get_attribute("textContent").strip()  
            playerstats["UD"] =                 stats[14].get_attribute("textContent").strip()  
            playerstats["FKD"] =                stats[15].get_attribute("textContent").strip()  
            playerstats["FK"] =                 stats[16].get_attribute("textContent").strip() 
            playerstats["FD"] =                 stats[17].get_attribute("textContent").strip() 
            playerstats["FK as T"] =            stats[18].get_attribute("textContent").strip()  
            playerstats["FD as T"] =            stats[19].get_attribute("textContent").strip()  
            playerstats["FK as CT"] =           stats[20].get_attribute("textContent").strip()  
            playerstats["FD as CT"] =           stats[21].get_attribute("textContent").strip()  
            playerstats["Trade K"] =            stats[22].get_attribute("textContent").strip()  
            playerstats["Trade D"] =            stats[23].get_attribute("textContent").strip()  
            playerstats["Trade FK"] =           stats[24].get_attribute("textContent").strip()  
            playerstats["Trade FD"] =           stats[25].get_attribute("textContent").strip()  
            playerstats["Trade FK as T"] =      stats[26].get_attribute("textContent").strip()  
            playerstats["Trade FD as T"] =      stats[27].get_attribute("textContent").strip()  
            playerstats["Trade FK as CT"] =     stats[28].get_attribute("textContent").strip()  
            playerstats["Trade FD as CT"] =     stats[29].get_attribute("textContent").strip()  
            playerstats["1vX"] =                stats[30].get_attribute("textContent").strip()  
            playerstats["1v5"] =                stats[31].get_attribute("textContent").strip()  
            playerstats["1v4"] =                stats[32].get_attribute("textContent").strip()  
            playerstats["1v3"] =                stats[33].get_attribute("textContent").strip()  
            playerstats["1v2"] =                stats[34].get_attribute("textContent").strip()  
            playerstats["1v1"] =                stats[35].get_attribute("textContent").strip()  
            playerstats["3K+"] =                stats[36].get_attribute("textContent").strip()  
            playerstats["5K"] =                 stats[37].get_attribute("textContent").strip()  
            playerstats["4K"] =                 stats[38].get_attribute("textContent").strip()  
            playerstats["3K"] =                 stats[39].get_attribute("textContent").strip()  
            playerstats["2K"] =                 stats[40].get_attribute("textContent").strip()  
            playerstats["1K"] =                 stats[41].get_attribute("textContent").strip()  
        
            # print(playerstats["Team Score"])
            match.append(playerstats)
        return match
        
    else :
        return []


In [77]:
def multiFetch(offset):
    
    
    df = pd.DataFrame()

    # frame = []

    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    driver = undetected_chromedriver.Chrome(options= options)

    # driver.implicitly_wait(0.1)
    # driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    for i in range(1):
        res = fetchMatch((offset + i),driver)
        # sleep(1)
        # print(res)
        df = pd.concat([pd.DataFrame(res),df])
        # driver.execute_script("window.open();")
        # driver.close()
        # driver.switch_to.window(driver.window_handles[-1])
        

    driver.quit()
    # print(frame)
    # df = pd.concat(frame)
    # display(df)
    return df

In [78]:
def testing(offset):
    print(offset)

In [79]:

        
df = pd.DataFrame()

nb = 10

nbarray = []

multiFetch(79000000)

,Match ID,Player ID,Rank,Rank Gain,Team,Team 1 Score,Team 2 Score,K,D,A,...,1v4,1v3,1v2,1v1,3K+,5K,4K,3K,2K,1K
0,79000000,7656119916601647,4,0,1,8,16,17,19,6,...,0,0,0,1,1,0,0,1,5,4
1,79000000,7656119914008402,7,0,1,8,16,16,19,7,...,0,0,0,0,1,0,1,0,2,8
2,79000000,7656119822018823,3,0,1,8,16,10,20,3,...,0,1,0,0,1,1,0,0,1,3
3,79000000,7656119917979524,2,0,1,8,16,11,20,3,...,0,0,0,0,1,0,0,1,2,4
4,79000000,7656119834332082,2,0,1,8,16,11,21,3,...,0,0,0,0,0,0,0,0,2,7
5,79000000,7656119839413884,11,0,2,8,16,25,15,4,...,0,0,0,0,2,0,0,2,4,11
6,79000000,7656119895046006,6,0,2,8,16,24,15,1,...,0,0,0,0,2,0,0,2,4,10
7,79000000,7656119825842951,7,0,2,8,16,19,14,1,...,0,0,0,0,0,0,0,0,4,11
8,79000000,7656119825996583,6,1,2,8,16,13,8,1,...,0,0,1,3,0,0,0,0,3,7
9,79000000,7656119820252568,4,1,2,8,16,17,15,7,...,0,0,0,0,0,0,0,0,3,11


In [80]:

display(df)
df.to_csv("test.csv")

""
